Drive link : https://drive.google.com/drive/folders/1gMvs6jq9J2hZLcrXm57bAVspNuuk1eff?usp=sharing

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
import os
from IPython.display import Image , display
from IPython import display
display.clear_output()

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
import cv2
from ultralytics import YOLO , solutions

model = YOLO('YOLO_Model.pt')
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_r

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
video_path = 'YOLO_Video.mp4'
cap =  cv2.VideoCapture(video_path)

In [ ]:
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video successfully opened.")

Video successfully opened.


## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(height)
print(width)
print(fps)

1080
1920
29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = 'output_video_yolo.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path , fourcc , fps , (width,height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
# Drawing settings
color = (0, 255, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 1


# ROIs and line positions
roi_top = (0, 0, width, height // 2)
roi_bottom = (0, height // 2, width, height)
roi_left = (0, 0, width // 2, height)
roi_right = (width // 2, 0, width, height)
line_top = height // 4
line_bottom = 3 * height // 4
line_left = width // 4
line_right = 3 * width // 4

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break


    unique_ids = set()
    unique_ids_top = set()
    unique_ids_bottom = set()
    unique_ids_left = set()
    unique_ids_right = set()

    results = model.track(frame, persist=False)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy().astype(int)
            obj_id = int(obj.id[0].cpu().numpy())
            class_id = int(obj.cls[0].cpu().numpy())
            confidence = float(obj.conf[0].cpu().numpy())

            unique_ids.add(obj_id)
            x1, y1, x2, y2 = bbox
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)


            label = f'ID: {obj_id} Conf: {confidence:.2f} Class: {class_id}'
            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)
            cv2.circle(frame, (center_x, center_y), 3, color, -1)

            # Update unique IDs by ROI
            if roi_top[1] <= center_y <= roi_top[3]:
                unique_ids_top.add(obj_id)
            elif roi_bottom[1] <= center_y <= roi_bottom[3]:
                unique_ids_bottom.add(obj_id)
            if roi_left[0] <= center_x <= roi_left[2]:
                unique_ids_left.add(obj_id)
            elif roi_right[0] <= center_x <= roi_right[2]:
                unique_ids_right.add(obj_id)


    cv2.line(frame, (0, line_top), (width, line_top), (255, 0, 0), 2)
    cv2.line(frame, (0, line_bottom), (width, line_bottom), (255, 0, 0), 2)
    cv2.line(frame, (line_left, 0), (line_left, height), (255, 0, 0), 2)
    cv2.line(frame, (line_right, 0), (line_right, height), (255, 0, 0), 2)

    cv2.putText(frame, (10, 30), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'Top Exits: {len(unique_ids_top)}', (10, height - 80), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'Bottom Exits: {len(unique_ids_bottom)}', (10, height - 60), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'Left Exits: {len(unique_ids_left)}', (10, height - 40), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'Right Exits: {len(unique_ids_right)}', (10, height - 20), font, font_scale, color, font_thickness, cv2.LINE_AA)


    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 61.4ms
Speed: 2.4ms preprocess, 61.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 40.1ms
Speed: 2.6ms preprocess, 40.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 39.0ms
Speed: 4.4ms preprocess, 39.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 38.9ms
Speed: 2.1ms preprocess, 38.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 81 cars, 1 truck, 2 vans, 38.9ms
Speed: 2.1ms preprocess, 38.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 80 cars, 1 truck, 2 vans, 38.9ms
Speed: 2.0ms preprocess, 38.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 39.0ms
Speed: 3.0ms preprocess, 39.0ms inference,

## Save and Submit Your Work

Submit both the notebook and the output video